In [1]:
from sklearn.datasets import make_classification
import torch

In [2]:
X,y = make_classification(
    n_samples = 10,
    n_features= 2,
    n_redundant=0,
    n_informative=2,
    n_classes=2,
    random_state=42,

)

In [3]:
X

array([[ 1.06833894, -0.97007347],
       [-1.14021544, -0.83879234],
       [-2.8953973 ,  1.97686236],
       [-0.72063436, -0.96059253],
       [-1.96287438, -0.99225135],
       [-0.9382051 , -0.54304815],
       [ 1.72725924, -1.18582677],
       [ 1.77736657,  1.51157598],
       [ 1.89969252,  0.83444483],
       [-0.58723065, -1.97171753]])

In [4]:
y

array([1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [5]:
X = torch.tensor(X,dtype=torch.float32)
y = torch.tensor(y,dtype=torch.long)

In [6]:
X

tensor([[ 1.0683, -0.9701],
        [-1.1402, -0.8388],
        [-2.8954,  1.9769],
        [-0.7206, -0.9606],
        [-1.9629, -0.9923],
        [-0.9382, -0.5430],
        [ 1.7273, -1.1858],
        [ 1.7774,  1.5116],
        [ 1.8997,  0.8344],
        [-0.5872, -1.9717]])

In [7]:
y

tensor([1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [9]:
from torch.utils.data import DataLoader, Dataset

In [10]:
class CustomDataset(Dataset):
  def __init__(self,features,labels):
    self.features = features
    self.labels = labels
  def __len__(self):
    return self.features.shape[0]
  def __getitem__(self,idx):
    return self.features[idx], self.labels[idx]

In [11]:
dataset = CustomDataset(X,y)

In [13]:
dataset[2]

(tensor([-2.8954,  1.9769]), tensor(0))

In [18]:
dataloader = DataLoader(dataset,batch_size=2,shuffle=True,num_workers=2)

In [19]:
for batch_features, batch_labels in dataloader:
  print(batch_features)
  print(batch_labels)
  print('--'*25)

tensor([[-1.1402, -0.8388],
        [ 1.7273, -1.1858]])
tensor([0, 1])
--------------------------------------------------
tensor([[ 1.8997,  0.8344],
        [-0.7206, -0.9606]])
tensor([1, 0])
--------------------------------------------------
tensor([[-2.8954,  1.9769],
        [ 1.7774,  1.5116]])
tensor([0, 1])
--------------------------------------------------
tensor([[-0.5872, -1.9717],
        [-0.9382, -0.5430]])
tensor([0, 1])
--------------------------------------------------
tensor([[ 1.0683, -0.9701],
        [-1.9629, -0.9923]])
tensor([1, 0])
--------------------------------------------------


In [22]:
import torch
from torch.utils.data import Dataset, DataLoader

# Small Dataset: y = 2x + 3 with small noise
torch.manual_seed(0)  # For reproducibility
x = torch.linspace(-5, 5, 100).reshape(-1, 1)
y = 2 * x + 3 + torch.randn(x.size()) * 0.5  # Adding noise


class SimpleRegressionDataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = x_data
        self.y_data = y_data

    def __len__(self):
        return len(self.x_data)

    def __getitem__(self, idx):
        return self.x_data[idx], self.y_data[idx]

# Create Dataset
dataset = SimpleRegressionDataset(x, y)
print(f"Dataset size: {len(dataset)}")
print("Sample 0:", dataset[0])


Dataset size: 100
Sample 0: (tensor([-5.]), tensor([-7.5629]))


In [23]:
# DataLoader with Mini-batch
batch_size = 16
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Check a Batch
for batch_x, batch_y in dataloader:
    print("Batch X Shape:", batch_x.shape)
    print("Batch Y Shape:", batch_y.shape)
    break  # Just show 1 batch


Batch X Shape: torch.Size([16, 1])
Batch Y Shape: torch.Size([16, 1])


In [24]:
import torch.nn as nn
import torch.optim as optim

class SimpleRegressor(nn.Module):
    def __init__(self):
        super(SimpleRegressor, self).__init__()
        self.linear = nn.Linear(1, 1)  # Simple Linear Regression

    def forward(self, x):
        return self.linear(x)


In [25]:
# Model
model = SimpleRegressor()

# Loss and Optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training Loop
epochs = 100
for epoch in range(epochs):
    for batch_x, batch_y in dataloader:
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}")


Epoch [10/100], Loss: 0.5172
Epoch [20/100], Loss: 0.1202
Epoch [30/100], Loss: 0.1982
Epoch [40/100], Loss: 0.4317
Epoch [50/100], Loss: 0.2848
Epoch [60/100], Loss: 0.4442
Epoch [70/100], Loss: 0.9227
Epoch [80/100], Loss: 0.3775
Epoch [90/100], Loss: 0.1212
Epoch [100/100], Loss: 0.1594


In [26]:
print("Learned parameters:")
for name, param in model.named_parameters():
    print(name, param.data)


Learned parameters:
linear.weight tensor([[1.9852]])
linear.bias tensor([3.0193])
